In [0]:
# Support Ticket Model  
# Classify IT request to which category it belongs.

In [0]:
# Mount google drive

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
# To download file

from google.colab import files

files.upload()

# files.download('sample_data/support_ticket_model.pkl')

# List directory
# files.os.listdir()

{}

In [0]:
# Import required modules here

import pandas as pd
# Data analysis and manipulation tool

from sklearn.feature_extraction.text import CountVectorizer
# Convert a collection of text documents to a matrix of token counts.

# The most straightforward one, it counts the number of times a token shows up 
# in the document and uses this value as its weight.

# The CountVectorizer provides a simple way to both tokenize a collection of text 
# documents and build a vocabulary of known words, but also to encode new documents using that vocabulary.

from sklearn import preprocessing
# The sklearn.preprocessing package provides several common utility functions and transformer classes 
# to change raw feature vectors into a representation that is more suitable for the downstream estimators.

import numpy as np
# NumPy is a library for the Python programming language, adding support for large,
# multi-dimensional arrays and matrices, along with a large collection of high-level 
# mathematical functions to operate on these arrays.

from sklearn import svm
# Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

# The linear SVM classifier works by drawing a straight line between two classes.
# https://towardsdatascience.com/support-vector-machine-python-example-d67d9b63f1c8

from sklearn.model_selection import StratifiedShuffleSplit
# This cross-validation object is a merge of StratifiedKFold and ShuffleSplit,
# which returns stratified randomized folds. The folds are made by preserving the percentage of samples for each class.

from sklearn import metrics
# The sklearn. metrics module implements several loss, score, and utility functions to measure classification performance.

import pickle
# The pickle module implements binary protocols for serializing and de-serializing a Python object structure. 

In [0]:
# Read and check data
df = pd.read_csv('latest_ticket_data-1.csv')

In [0]:
df.head()

,Description,Category
0,hi since recruiter lead permission approve req...,Application
1,re expire days hi ask help update passwords co...,Application
2,verification warning hi has got attached pleas...,Application
3,please dear looks blacklisted receiving mails ...,Application
4,dear modules report report cost thank much reg...,Application


In [0]:
# Check for null value
df.isnull().sum()

Description    0
Category       0
dtype: int64

In [0]:
# Use sklearn to make feature vectorizer.

# Feature extraction is very different from Feature selection: the former consists in transforming arbitrary data,
# such as text or images, into numerical features usable for machine learning.

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Description'])

In [0]:
type(X)

scipy.sparse.csr.csr_matrix

In [0]:
# Use labelencorder for class colum

le = preprocessing.LabelEncoder()
# Sklearn provides a very efficient tool for encoding the levels of categorical features into numeric values.
# LabelEncoder encode labels with a value between 0 and n_classes-1 where n is the number of distinct labels.

zz= le.fit(df['Category'])  
# Fit label encoder

print(le.classes_)
# Holds the label for each class.


['Application' 'Database' 'Network' 'Security' 'User Maintenance']


In [0]:
y = le.fit_transform(df['Category'])
# Fit label encoder and return encoded labels

In [0]:
type(y)

numpy.ndarray

In [0]:
# Use StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2,random_state=0)
# https://kite.com/python/docs/sklearn.model_selection.StratifiedShuffleSplit

# n_splits:- Number of re-shuffling & splitting iterations.

# test_size:- If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split.
# If train_size is also None, it will be set to 0.1.

# random_state:- If you don't specify the random_state in the code, then every time you run(execute)
# your code a new random value is generated and the train and test datasets would have different values each time.

In [0]:
for train_index, test_index in sss.split(X, y): # Generate indices to split data into training and test set.
     #print("TRAIN:", train_index, "TEST:", test_index, end="=========")
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
     test_df = df.iloc[test_index, :] # To copy test data in different file.


In [0]:
test_df.head()

,Description,Category
1294,re secured area tower approved thank officer n...,User Maintenance
670,re please tickets assign tickets please trees ...,Database
1933,door open floor tower dear could you please fi...,Security
1203,re card va se,User Maintenance
65,old hi please old breakdown thank officer,Application


In [0]:
# Save test file as csv

test_df.to_csv('test.csv')

In [0]:
# Build support vector machine model
clf = svm.SVC(kernel='linear')

# Use linear kernel when number of features is larger than number of observations.
# Use gaussian kernel when number of observations is larger than number of features.
# If number of observations is larger than 50,000 speed could be an issue when using gaussian kernel; hence, one might want to use linear kernel

In [0]:
# Train the model using the training sets
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
# Predict the response for test dataset
y_pred = clf.predict(X_test)

In [0]:
# Evaluating the Model

# Accuracy can be computed by comparing actual test set values and predicted values.
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Accuracy classification score.
# In multilabel classification, this function computes subset accuracy: 
# the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.

Accuracy: 0.6983333333333334


In [0]:
# Save the model
filename = 'support_ticket_model.pkl'
pickle.dump(clf, open(filename, 'wb')) # Write the pickled representation of the object obj to the open file object file.

In [0]:
# Save CountVectorizer Model
CountVectorizerModel = 'CountVectorizerModel.pkl'
pickle.dump(vectorizer, open(CountVectorizerModel, 'wb'))

In [0]:
# Save Label encoder Model
labelEncorderModel = 'labelEncorderModel.pkl'
pickle.dump(le, open(labelEncorderModel, 'wb'))